# Test the coverage of the GLRT cutoff value
Recall, we come up with an empirical log-likelihood threshold, that we believe covers the log likelihood of the true parameter at least 1-alpha fraction of the time.
Test this claim by seeing whether the log-likelihood of the true parameter is in fact above the threshold 95% of the time.

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [8]:
from uncertainty.DataGeneration import linearRegression_normal
from uncertainty.glrt_stat import bootstrapGLRTcis
from uncertainty.torch_linear import TorchLinear

In [4]:
def MSE(y, yPred):
    return np.mean((y - yPred)**2)

In [24]:
numTrials = 1000
covered = []

beta = np.array([1])
n = 100
    
for _ in range(numTrials):
    # Get data
    X, y = linearRegression_normal(beta=np.array(beta), cov=np.eye(len(beta)), sigma=1, n=n)

    # Fit a model
    LR = LinearRegression()
    LR.fit(X, y)

    nllBound = bootstrapGLRTcis(modelFn=LinearRegression,
                                X=X,
                                y=y,
                                nllFn=MSE, 
                                alpha=0.05, 
                                replicates=1000)
    
    # Get the MSE of betaStar
    yPredStar = np.dot(X, beta)
    
    covered.append(MSE(y, yPredStar) <= nllBound)

print(np.mean(covered))

0.948


In [25]:
numTrials = 100

beta = np.array([1])
n = 100
alpha = 0.2   # Use a much larger alpha so we don't have to run quite so many trials

for modelFn in [LinearRegression, TorchLinear]:
    covered = []
    for _ in range(numTrials):
        # Get data
        X, y = linearRegression_normal(beta=np.array(beta), cov=np.eye(len(beta)), sigma=1, n=n)

        # Fit a model
        LR = LinearRegression()
        LR.fit(X, y)

        nllBound = bootstrapGLRTcis(modelFn=LinearRegression,
                                    X=X,
                                    y=y,
                                    nllFn=MSE, 
                                    alpha=alpha, 
                                    replicates=100)

        # Get the MSE of betaStar
        yPredStar = np.dot(X, beta)

        covered.append(MSE(y, yPredStar) <= nllBound)
    
    print(np.mean(covered))

0.82
0.79
